In [1]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
df_train = pd.read_csv("cityu10c_train_dataset.csv")
df_test = pd.read_csv("cityu10c_test_dataset.csv")
label_col = "LoanApproved"

In [3]:
# Xóa các cột không cần thiết
df_train.drop(columns=["ApplicationDate"], errors="ignore", inplace=True)
df_test.drop(columns=["ApplicationDate"], errors="ignore", inplace=True)

# Xử lý giá trị thiếu
for col in df_train.columns:
    if df_train[col].dtype == "object":
        df_train[col] = df_train[col].fillna(df_train[col].mode()[0])
    else:
        df_train[col] = df_train[col].fillna(df_train[col].median())

for col in df_test.columns:
    if df_test[col].dtype == "object":
        df_test[col] = df_test[col].fillna(df_test[col].mode()[0])
    else:
        df_test[col] = df_test[col].fillna(df_test[col].median())

# Định nghĩa cột số và cột phân loại
categorical_features = ["EmploymentStatus", "EducationLevel", "MaritalStatus", "HomeOwnershipStatus", "LoanPurpose"]
numerical_features = ["Age", "AnnualIncome", "CreditScore", "LoanAmount", "LoanDuration"]

# Xây dựng pipeline xử lý dữ liệu
numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Tạo pipeline mô hình
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

# Chia dữ liệu train và validation
X = df_train.drop(columns=[label_col])
y = df_train[label_col]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Lưu pipeline vào file
with open('xgboost_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

# Dự đoán trên tập test
df_test_pred = pipeline.predict(df_test.drop(columns=["ID"], errors="ignore"))

# Xuất file kết quả
df_test_ids = df_test["ID"]
submission = pd.DataFrame({"ID": df_test_ids, "LoanApproved": df_test_pred})
submission.to_csv("submission.csv", index=False)

print("Dự đoán hoàn tất.")


Dự đoán hoàn tất.
